# Bayesian Personalized Recommendation

The code in this notebook defines the Bayesian Personalized Ranking model and shows how to train it on data in the format: (Target ISIN, Better Recommendation ISIN, Worse Recommendation ISIN).

Please see BPRDataPreparation.ipynb for an example of how to bootstrap this model by creating training data from the outputs of a distance-based model.

In [2]:
# THIS FILE CONTAINS A MAPPING BETWEEN ISIN AND AN INDEX FOR THAT BOND THAT THE MODEL WILL USE TO IDENTIFY IT
# e.g. {"US00037BAB80": 0, "US00037BAC63": 1, ...}
isin_to_index_mapping_file = 'isin_to_index2.json'

In [4]:
# A saved PyTorch Tensor that contains rows of (TARGET BOND INDEX, BETTER RECOMMENDATION INDEX, WORSE RECOMMENDATION INDEX)
training_data_tensor = 'train/all_rankings.pt'

In [6]:
# NOTE: The PyTorch Tensor is faster, so load it first

# A saved CSV that contains rows of (TARGET BOND ISIN, BETTER RECOMMENDATION ISIN, WORSE RECOMMENDATION ISIN)
training_data_csv = 'train/all_rankings.csv'

In [1]:
import csv
import json
import numpy as np
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from data import SingleDayDataLoader

In [ ]:
class BPRModel(nn.Module):
    """
    This class implements a PyTorch model for Bayesian Personalized Recommendation
    Per https://arxiv.org/abs/1205.2618
    """
    def __init__(self, num_bonds, num_factors, regularization_lambda=0.0000001):
        super().__init__()
        
        self.regularization_lambda = regularization_lambda
        self.bond_embedding = nn.Embedding(num_bonds, num_factors)
    
    def forward(self, rankings):
        # Input is a batch of (bond, better recommendation, worse recommendation) triplets
        bonds = rankings[:,0]
        better_recommendations = rankings[:,1]
        worse_recommendations = rankings[:,2]
        
        # Fetch our existing latent vector representation of each bond
        bond_embeddings = self.bond_embedding(bonds)
        better_recommendation_embeddings = self.bond_embedding(better_recommendations)
        worse_recommendation_embeddings = self.bond_embedding(worse_recommendations)
        
        # Compute the dot product of the latent embedding
        # We equate large dot products with high bond similarity
        dot_better = torch.sum(bond_embeddings * better_recommendation_embeddings, dim=1)
        dot_worse = torch.sum(bond_embeddings * worse_recommendation_embeddings, dim=1)
        dot_diff = dot_better - dot_worse
        
        log_likelihood = torch.mean(F.logsigmoid(dot_diff))
        
        # useful to track how many the model "got right", i.e. agrees that better ones are better
        auc = torch.mean((dot_diff > 0).float())
        
        # Recall that a guassian prior is equivalent to l2 regularization
        # http://bjlkeng.github.io/posts/probabilistic-interpretation-of-regularization/
        prior = sum(
            [
                self.regularization_lambda * torch.sum(bond_embeddings * bond_embeddings),
                lambda_item_bond * torch.sum(better_recommendation_embeddings * better_recommendation_embeddings),
                lambda_item_bond * torch.sum(worse_recommendation_embeddings * worse_recommendation_embeddings),
            ]
        )
        
        return log_likelihood, prior, auc        

In [ ]:
class ModelHelper(object):
    def __init__(self, model, isin_to_index_mapping, metadata):
        
        self.isin_to_index = isin_to_index_mapping
        self.index_to_isin = {idx: isin for isin, idx in self.isin_to_index.items()}
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model
        self.metadata = metadata
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.005)
        
    def predict(self, isin, n=10):
        with torch.no_grad():
            bond_idx = self.isin_to_index[isin]
            bond = self.model.bond_embedding(torch.tensor(bond_idx, device=self.device))
            dots = torch.sum(bond * self.model.bond_embedding.weight.data, dim=1)
            bond_indices = torch.argsort(dots, descending=True)[:n]
            isins = [self.index_to_isin[i] for i in bond_indices.cpu().numpy().tolist()]
            return isins
    
    def process_feedback(self, feedback):
        # Save a list of [(bond, better recommendation, worse recommendation), ...]
        feedback = [[self.isin_to_index[isin] for isin in isin_triplet] for isin_triplet in feedback]
        feedback = torch.tensor(feedback, device=self.device)
        likelihood, prior, auc = self.model(feedback)
        loss = -likelihood + prior
        loss.backward()
        self.optimizer.step()
    
    def display(self, isins, display_cols=None):
        if display_cols is None:
            display_cols = ['BCLASS3', 'Ticker', 'Country', 'Bid Spread', 'Cur Yld', 'OAS', 'OAD', 'Cpn']
        print(
            tabulate(
                self.metadata.get_bonds(isins)[display_cols], 
                headers=display_cols, 
                #showindex="never"
            )
        )
    
    def save(self, path):
        torch.save(self.model.state_dict(), path)
    
    def load(self, path):
        self.model.load_state_dict(torch.load(path))
        
        

In [ ]:
class LoadTensorDataset(Dataset):
    """
    Load the dataset from a saved tensor file on disk
    """
    def __init__(self, tensor_file):
        self.data = torch.load(tensor_file)
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
epochs = 30
num_factors = 32 # Number of latent factors, i.e we represent each bond as a 32 dimensional vector

# Independent lambda regularization values 
# for user, items and bias.
regularization_lambda = 0.0000001

# Our initial learning rate 
lr = 0.0001

# How many (u,i,j) triplets we sample for each batch
samples = 15000

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset = LoadTensorDataset(training_data_tensor)

In [ ]:
isin_to_index_mapping = json.load(open(isin_to_index_mapping_file))

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
dataloader = DataLoader(train_dataset, shuffle=True, num_workers=6, batch_size=samples)

In [ ]:
num_bonds = len(isin_to_index_mapping)
model = BPRModel(num_bonds, num_factors)

In [ ]:
# model.load('models/bpr_v1.pt')

In [ ]:
model = model.to(device)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [ ]:
model_helper = ModelHelper(model, isin_to_index_mapping, metadata=SingleDayDataLoader())

In [ ]:
test_bond = 'US539830AY52'

In [ ]:
for epoch in range(epochs):
    exp_lr_scheduler.step()
    for batch_idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        batch = batch.to(device)
        likelihood, prior, auc = model(batch)
        loss = -likelihood + prior
        loss.backward()
        optimizer.step()
        if not batch_idx % 200:
            print("Epoch: {}, Batch: {}, Loss: {}, AUC: {}".format(epoch, batch_idx, loss, auc))
        if not batch_idx % 500:
            print("\nTest bond:")
            model_helper.display([test_bond])
            print("\nSample predictions:")
            model_helper.display(
                model_helper.predict(test_bond)
            )

In [ ]:
model.save('models/bpr_v2.pt')

### Incorporate user feedback

In [ ]:
# Get existing predictions

model_helper.display(
    model_helper.predict(test_bond)
)

In [ ]:
# Choose a pair that should be reorder

model_helper.process_feedback([(test_bond, 'US92340LAC37', 'US565122AG31')])

In [ ]:
# See the new results! Note, you might have to run feedback a few times in order to have the desired effect

model_helper.display(
    model_helper.predict(test_bond)
)